# Driver Performance Prediction
## Extract Data

In [894]:
# pip install pandas
# pip install matplotlib
# pip install sqlite3
# pip install kaggle
# move kaggle.json file into ~/.kaggle (mac) or C:\Users<Windows-username>.kaggle\kaggle.json (windows)

import kaggle
import pandas as pd
import numpy as np
import matplotlib as plt
import sqlite3 as db

dataset = "rohanrao/formula-1-world-championship-1950-2020"

kaggle.api.dataset_download_files(dataset, path='./', unzip=True)

print(f"Dataset {dataset} downloaded successfully!")

Dataset URL: https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020
Dataset rohanrao/formula-1-world-championship-1950-2020 downloaded successfully!


In [895]:
results_df = pd.read_csv('results.csv')
qualifying_df = pd.read_csv('qualifying.csv')
lap_times_df = pd.read_csv('lap_times.csv')
pit_stops_df = pd.read_csv('pit_stops.csv')
driver_standings_df = pd.read_csv('driver_standings.csv')
races_df = pd.read_csv('races.csv')
constructors_df = pd.read_csv('constructors.csv')
status_df = pd.read_csv('status.csv')

## Transform Data
Changing data types, dropping and renaming columns

In [896]:

results_df.drop(['number', 'positionText', 'time', 'rank', 'fastestLap', 'fastestLapTime', 'fastestLapSpeed', 'laps', 'points', 'positionOrder', 'grid'], axis=1, inplace=True)

results_df.rename(columns={'position': 'final_position'}, inplace=True)

results_df['final_position'] = results_df['final_position'].replace('\\N', np.nan)
results_df.final_position = pd.to_numeric(results_df.final_position, errors='coerce')
results_df.milliseconds = pd.to_numeric(results_df.milliseconds, errors='coerce')
print(results_df.dtypes)
results_df.head()

resultId            int64
raceId              int64
driverId            int64
constructorId       int64
final_position    float64
milliseconds      float64
statusId            int64
dtype: object


,resultId,raceId,driverId,constructorId,final_position,milliseconds,statusId
0,1,18,1,1,1.0,5690616.0,1
1,2,18,2,2,2.0,5696094.0,1
2,3,18,3,3,3.0,5698779.0,1
3,4,18,4,4,4.0,5707797.0,1
4,5,18,5,1,5.0,5708630.0,1


In [897]:
qualifying_df.drop(['number', 'q1', 'q2', 'q3', 'qualifyId'], axis=1, inplace=True)
qualifying_df.rename(columns={'position': 'qualifying_position'}, inplace=True)

print(qualifying_df.dtypes)
qualifying_df.head()

raceId                 int64
driverId               int64
constructorId          int64
qualifying_position    int64
dtype: object


,raceId,driverId,constructorId,qualifying_position
0,18,1,1,1
1,18,9,2,2
2,18,5,1,3
3,18,13,6,4
4,18,2,2,5


In [898]:
lap_times_df.drop(['time', 'lap', 'position'], axis=1, inplace=True)
lap_times_df.rename(columns={'milliseconds': 'lap_time_milliseconds'}, inplace=True)

print(lap_times_df.dtypes)
lap_times_df.head()

raceId                   int64
driverId                 int64
lap_time_milliseconds    int64
dtype: object


,raceId,driverId,lap_time_milliseconds
0,841,20,98109
1,841,20,93006
2,841,20,92713
3,841,20,92803
4,841,20,92342


In [899]:
pit_stops_df.drop(['milliseconds', 'time', 'lap'], axis=1, inplace=True)
pit_stops_df.rename(columns={'duration': 'stop_duration', 'stop':'pit_stop'}, inplace=True)

pit_stops_df.stop_duration = pd.to_numeric(pit_stops_df.stop_duration, errors='coerce')
print(pit_stops_df.dtypes)
pit_stops_df.head()

raceId             int64
driverId           int64
pit_stop           int64
stop_duration    float64
dtype: object


,raceId,driverId,pit_stop,stop_duration
0,841,153,1,26.898
1,841,30,1,25.021
2,841,17,1,23.426
3,841,4,1,23.251
4,841,13,1,23.842


In [900]:
driver_standings_df.drop(['position','positionText', 'wins', 'driverStandingsId'], axis=1, inplace=True)
driver_standings_df.rename(columns={'points': 'driver_points'}, inplace=True)

print(driver_standings_df.dtypes)
driver_standings_df.head()

raceId             int64
driverId           int64
driver_points    float64
dtype: object


,raceId,driverId,driver_points
0,18,1,10.0
1,18,2,8.0
2,18,3,6.0
3,18,4,5.0
4,18,5,4.0


In [901]:
races_df.drop(['date','time', 'round','url', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time'], axis=1, inplace=True)
races_df.rename(columns={'name': 'circuit_name'}, inplace=True)

print(races_df.dtypes)
#filtered_races_df = races_df[races_df['year'] >= 2020]
#filtered_races_df.head()

raceId           int64
year             int64
circuitId        int64
circuit_name    object
dtype: object


In [902]:
constructors_df.drop(['url', 'nationality', 'constructorRef'], axis=1, inplace=True)
constructors_df.rename(columns={'name': 'constructor_name'}, inplace=True)

print(constructors_df.dtypes)
constructors_df.head()

constructorId        int64
constructor_name    object
dtype: object


,constructorId,constructor_name
0,1,McLaren
1,2,BMW Sauber
2,3,Williams
3,4,Renault
4,5,Toro Rosso


In [903]:
print(status_df.dtypes)
status_df.head()

statusId     int64
status      object
dtype: object


,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine


In [904]:
races_years = races_df['year'].unique()

lap_times_years = pd.merge(lap_times_df, races_df, on='raceId')['year'].unique()
pit_stops_years = pd.merge(pit_stops_df, races_df, on='raceId')['year'].unique()
qualifying_years = pd.merge(qualifying_df, races_df, on='raceId')['year'].unique()

races_years.sort(), lap_times_years.sort(), pit_stops_years.sort(), qualifying_years.sort()

(races_years, lap_times_years, pit_stops_years, qualifying_years)

(array([1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960,
        1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
        1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
        1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
        1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
        2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
        2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]),
 array([1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006,
        2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
        2018, 2019, 2020, 2021, 2022, 2023, 2024]),
 array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021,
        2022, 2023, 2024]),
 array([1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
        2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
        2016, 2017, 2018, 2019, 2020, 2021

## Merging data

In [905]:
# merge datasets together

merge1_df = pd.merge(results_df, races_df, on='raceId', how='inner')
merge1_df = merge1_df.drop_duplicates(subset=['raceId', 'driverId'])

merge2_df = pd.merge(merge1_df, constructors_df, on='constructorId', how='inner')
merge2_df = merge2_df.drop_duplicates(subset=['raceId', 'driverId'])

merge3_df = pd.merge(merge2_df, driver_standings_df, on=['driverId', 'raceId'], how='inner')
merge3_df = merge3_df.drop_duplicates(subset=['raceId', 'driverId'])

merge4_df = pd.merge(merge3_df, lap_times_df, on=['driverId', 'raceId'], how='inner')
merge4_df = merge4_df.drop_duplicates(subset=['raceId', 'driverId'])

merge5_df = pd.merge(merge4_df, pit_stops_df, on=['driverId', 'raceId'], how='inner')
merge5_df = merge5_df.drop_duplicates(subset=['raceId', 'driverId'])

merge6_df = pd.merge(merge5_df, qualifying_df, on=['driverId', 'raceId'], how='inner')
merge6_df = merge6_df.drop_duplicates(subset=['raceId', 'driverId'])

combined_df = pd.merge(merge6_df, status_df, on='statusId', how='inner')
combined_df = combined_df.drop_duplicates(subset=['raceId', 'driverId'])

duplicates_check = combined_df.duplicated(subset=['raceId', 'driverId']).any()
print(duplicates_check)

display(combined_df)


False


,resultId,raceId,driverId,constructorId_x,final_position,milliseconds,statusId,year,circuitId,circuit_name,constructor_name,driver_points,lap_time_milliseconds,pit_stop,stop_duration,constructorId_y,qualifying_position,status
0,20779,841,20,9,1.0,5370259.0,1,2011,1,Australian Grand Prix,Red Bull,25.0,98109,1,22.603,9,1,Finished
1,20780,841,1,1,2.0,5392556.0,1,2011,1,Australian Grand Prix,McLaren,18.0,100573,1,23.227,1,2,Finished
2,20781,841,808,4,3.0,5400819.0,1,2011,1,Australian Grand Prix,Renault,15.0,102835,1,24.535,4,6,Finished
3,20782,841,4,6,4.0,5402031.0,1,2011,1,Australian Grand Prix,Ferrari,12.0,106144,1,23.251,6,5,Finished
4,20783,841,17,9,5.0,5408430.0,1,2011,1,Australian Grand Prix,Red Bull,10.0,101467,1,23.426,9,3,Finished
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5300,26500,1131,822,15,16.0,NaN,11,2024,70,Austrian Grand Prix,Sauber,0.0,80918,1,21.918,15,18,+1 Lap
5301,26501,1131,855,15,17.0,NaN,11,2024,70,Austrian Grand Prix,Sauber,0.0,84058,1,23.198,15,20,+1 Lap
5302,26502,1131,4,117,18.0,NaN,11,2024,70,Austrian Grand Prix,Aston Martin,41.0,79908,1,21.483,117,15,+1 Lap
5303,26503,1131,858,3,19.0,NaN,12,2024,70,Austrian Grand Prix,Williams,0.0,84605,1,31.722,3,19,+2 Laps


# Filter and drop nulls
Only show drives who have finished a race

In [906]:
combined_df.drop(['constructorId_y'], axis=1, inplace=True)
combined_df.rename(columns={'constructorId_x': 'constructorId'}, inplace=True)

df = combined_df[combined_df['status'] == 'Finished']

df = df.dropna()
df.isnull().sum()
display(df)

,resultId,raceId,driverId,constructorId,final_position,milliseconds,statusId,year,circuitId,circuit_name,constructor_name,driver_points,lap_time_milliseconds,pit_stop,stop_duration,qualifying_position,status
0,20779,841,20,9,1.0,5370259.0,1,2011,1,Australian Grand Prix,Red Bull,25.0,98109,1,22.603,1,Finished
1,20780,841,1,1,2.0,5392556.0,1,2011,1,Australian Grand Prix,McLaren,18.0,100573,1,23.227,2,Finished
2,20781,841,808,4,3.0,5400819.0,1,2011,1,Australian Grand Prix,Renault,15.0,102835,1,24.535,6,Finished
3,20782,841,4,6,4.0,5402031.0,1,2011,1,Australian Grand Prix,Ferrari,12.0,106144,1,23.251,5,Finished
4,20783,841,17,9,5.0,5408430.0,1,2011,1,Australian Grand Prix,Red Bull,10.0,101467,1,23.426,3,Finished
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5292,26492,1131,825,210,8.0,5123153.0,1,2024,70,Austrian Grand Prix,Haas F1 Team,5.0,78826,1,21.518,12,Finished
5293,26493,1131,817,215,9.0,5123967.0,1,2024,70,Austrian Grand Prix,RB F1 Team,11.0,79507,1,21.502,11,Finished
5294,26494,1131,842,214,10.0,5124564.0,1,2024,70,Austrian Grand Prix,Alpine F1 Team,6.0,79109,1,21.431,13,Finished
5295,26495,1131,844,6,11.0,5129854.0,1,2024,70,Austrian Grand Prix,Ferrari,150.0,82085,1,31.917,6,Finished


# Exploratory Data Analysis (EDA)

In [907]:
print(df.dtypes)
display(df.describe())

resultId                   int64
raceId                     int64
driverId                   int64
constructorId              int64
final_position           float64
milliseconds             float64
statusId                   int64
year                       int64
circuitId                  int64
circuit_name              object
constructor_name          object
driver_points            float64
lap_time_milliseconds      int64
pit_stop                   int64
stop_duration            float64
qualifying_position        int64
status                    object
dtype: object


,resultId,raceId,driverId,constructorId,final_position,milliseconds,statusId,year,circuitId,driver_points,lap_time_milliseconds,pit_stop,stop_duration,qualifying_position
count,2787.000000,2787.000000,2787.000000,2787.000000,2787.000000,2.787000e+03,2787.0,2787.000000,2787.000000,2787.000000,2787.000000,2787.000000,2787.000000,2787.000000
mean,23720.951561,985.928597,506.953714,59.529602,6.582705,5.886109e+06,1.0,2017.405095,25.673125,78.324363,108775.733764,1.028346,24.288727,8.100108
std,1701.486134,88.340493,396.930266,77.414752,4.102845,1.029226e+06,0.0,4.089700,25.221027,85.844709,20062.317904,0.188279,3.911071,5.392660
min,20779.000000,841.000000,1.000000,1.000000,1.000000,4.421143e+06,1.0,2011.000000,1.000000,0.000000,68922.000000,1.000000,16.369000,1.000000
25%,22145.500000,900.500000,16.000000,6.000000,3.000000,5.339457e+06,1.0,2014.000000,7.000000,15.000000,95685.000000,1.000000,21.869000,4.000000
50%,23764.000000,988.000000,815.000000,9.000000,6.000000,5.687892e+06,1.0,2017.000000,15.000000,46.000000,105925.000000,1.000000,23.430000,7.000000
75%,25287.500000,1067.000000,832.000000,131.000000,9.000000,6.074536e+06,1.0,2021.000000,32.000000,116.000000,117644.500000,1.000000,25.397000,12.000000
max,26496.000000,1131.000000,860.000000,215.000000,19.000000,1.474314e+07,1.0,2024.000000,80.000000,575.000000,257359.000000,3.000000,56.710000,23.000000


Load into SQLite

In [908]:
conn = db.connect('database.db')
cursor = conn.cursor()

How many unique drivers, teams and circuits are represented in the dataset

How does the qualifying position influence the final race position

Which drivers are the most consistent across races

Which drivers gain or lose the most position during races


How does the number of pit stops affect the final race position

How does the constructor (team) influence driver performance?

In [909]:
conn.commit()
conn.close()